In [2]:
from threading import Thread
import random
import time
import sys, getopt
from datetime import datetime
import pypyodbc
import pandas as pd
import mysql.connector

In [3]:
length = 0
for x in pd.read_csv('Barcelona1/Barcelona1.csv',chunksize=500):
    x.to_csv("barcelona_sample.csv",index=False)
    length+= x.shape[0]
    break
length

500

In [4]:
pd.DataFrame(None,columns=x.columns).to_csv("barcelona_mr.csv")
i=1
for x in pd.read_csv('Barcelona1/Barcelona1.csv',chunksize=500):
    x.insert(0,"index",[i for y in range(len(x))])
    i=i+1
    x.fillna(-1,inplace=True)
    x.to_csv('barcelona_mr.csv', mode='a', header=False)

In [15]:
stmt = "INSERT INTO {} ({}) VALUES ({})".format("barcelona",','.join(list(x.columns)),
                                                       ','.join(["%s" for i in xrange(len(list(x.columns)))]))
stmt

'INSERT INTO barcelona (Source,City_x,CheckInDate,CheckOutDate,LOS,Occupancy,RoomType,BoardType,Hotel_Name_x,Star_Rating,Guest_Rating,Guest_Recommendation,Guest_Review_Count,Strike_Rate,Rate,Rooms_Left,Cancellation_Policy,Discount,Deal,Page_Position,Hotel_Rank,Hotel_Overall_Rank,Latitude_x,Longitude_x,Location,Phone_Number,Reward_Points,Star_Rating_x,Latitude_y,Longitude_y,Distance_From_Airport_Miles_x,Distance_From_Convention_Center_Miles_x,Distance_From_City_Center_Miles_x,TYPE,sub_type,Address,ZipCode,Zip2,Zips,lat,long,Density_1_mile_ALL,Density_2_miles_ALL,Density_3_mile_ALL,Density_5_miles_ALL,Rating,Star,No_Of_Review,Certificate_of_Excellence,Recommended_length_of_visit,Fee,No_of_rooms,Source_website,Density_1_mile_Private_Room,Density_2_miles_Private_Room,Density_3_mile_Private_Room,Density_5_miles_Private_Room,Density_1_mile_Entire_Home_Apt,Density_2_miles_Entire_Home_Apt,Density_3_mile_Entire_Home_Apt,Density_5_miles_Entire_Home_Apt,Density_1_mile_Shared_Room,Density_2_miles_

AttributeError: 'TextFileReader' object has no attribute 'index'

In [131]:
###### table creation

import itertools
table_string = []
dl = {"object": "VARCHAR(1000)",
         "float64": "float",
         "int64": "int"}
for m, n in itertools.izip(x.dtypes.keys(), x.dtypes):
    if m == 'long':
        m='lon'
    table_string.append("{} {}".format(m,dl[str(n)]))
stmt = "CREATE TABLE barcelona({})".format(','.join(table_string))
print stmt
cnx = mysql.connector.connect(user='puneet', password='root',host='192.157.250.82',database = 'testdb')
cursor = cnx.cursor()
cursor.execute(stmt)
cnx.close()

CREATE TABLE barcelona(Source VARCHAR(1000),City_x VARCHAR(1000),CheckInDate VARCHAR(1000),CheckOutDate VARCHAR(1000),LOS int,Occupancy int,RoomType VARCHAR(1000),BoardType VARCHAR(1000),Hotel_Name_x VARCHAR(1000),Star_Rating float,Guest_Rating float,Guest_Recommendation VARCHAR(1000),Guest_Review_Count float,Strike_Rate float,Rate float,Rooms_Left int,Cancellation_Policy VARCHAR(1000),Discount VARCHAR(1000),Deal float,Page_Position int,Hotel_Rank int,Hotel_Overall_Rank int,Latitude_x float,Longitude_x float,Location VARCHAR(1000),Phone_Number VARCHAR(1000),Reward_Points int,Star_Rating_x float,Latitude_y float,Longitude_y float,Distance_From_Airport_Miles_x float,Distance_From_Convention_Center_Miles_x float,Distance_From_City_Center_Miles_x float,TYPE float,sub_type float,Address float,ZipCode float,Zip2 float,Zips float,lat float,lon float,Density_1_mile_ALL float,Density_2_miles_ALL float,Density_3_mile_ALL float,Density_5_miles_ALL float,Rating float,Star float,No_Of_Review float,

ProgrammingError: 1050 (42S01): Table 'barcelona' already exists

In [ ]:
#### data push  function with multithreads
chunksize = 800000

def push_to_db(chunk, *args):
    tuples = [tuple(x) for x in chunk.values]
    print tuples[1:3]
    stmt = "INSERT INTO {} ({}) VALUES ({})".format("t2",','.join(list(chunk.columns)),
                                                       ','.join(["%s" for i in xrange(len(list(chunk.columns)))]))
    cnx = mysql.connector.connect(user='puneet', password='root',
                                  host='192.157.250.82',database = 'testdb')
    cursor = cnx.cursor()
    cursor.executemany(stmt, tuples)
    cnx.commit()
    cnx.close()
    print "process"

for chunk in pd.read_csv('Barcelona1/Barcelona1.csv', chunksize=chunksize):
    t = Thread(target=push_to_db,args=(chunk,))
    t.start()
    
print("Completed!")
print(datetime.now())

In [40]:
cnxn = pypyodbc.connect('DRIVER={SQL Server};SERVER=rgsvrsql-01.rategain.local\UHS;DATABASE=Azhar_Test;UID=sa;PWD=SA1234!@#$')
sql1 = "CREATE TABLE t2(name VARCHAR(100),num int)"
crsr.execute(sql1)
cnxn.commit()
crsr.close()
cnxn.close()

In [44]:
cnxn = pypyodbc.connect('DRIVER={SQL Server};SERVER=rgsvrsql-01.rategain.local\UHS;DATABASE=Azhar_Test;UID=sa;PWD=SA1234!@#$')
crsr = cnxn.cursor()
sql2 = """BULK INSERT t2 FROM 'C:\Users\puneet.jindal\Documents\test_db.csv' WITH (FIELDTERMINATOR = ',')"""
crsr.execute(sql2)
crsr.close()
cnxn.close()

ProgrammingError: (u'42000', u'[42000] [Microsoft][ODBC SQL Server Driver][SQL Server]Cannot bulk load because the file "C:\\Users\\puneet.jindal\\Documents\test_db.csv" could not be opened. Operating system error code 3(failed to retrieve text for this error. Reason: 15105).')

In [66]:
import mysql.connector
cnx = mysql.connector.connect(user='puneet', password='root',host='192.157.250.82',database = 'testdb')
stmt = "CREATE TABLE t2(name VARCHAR(100),num int)"
cursor = cnx.cursor()
cursor.execute(stmt)
cnx.close()


In [101]:
for x in pd.read_csv('Barcelona1/Barcelona1.csv',chunksize=2):
    #print list(x.columns)
    print x.dtypes.keys()
    print x.dtypes.values
    break

Index([u'Source', u'City_x', u'CheckInDate', u'CheckOutDate', u'LOS',
       u'Occupancy', u'RoomType', u'BoardType', u'Hotel_Name_x',
       u'Star_Rating', u'Guest_Rating', u'Guest_Recommendation',
       u'Guest_Review_Count', u'Strike_Rate', u'Rate', u'Rooms_Left',
       u'Cancellation_Policy', u'Discount', u'Deal', u'Page_Position',
       u'Hotel_Rank', u'Hotel_Overall_Rank', u'Latitude_x', u'Longitude_x',
       u'Location', u'Phone_Number', u'Reward_Points', u'Star_Rating_x',
       u'Latitude_y', u'Longitude_y', u'Distance_From_Airport_Miles_x',
       u'Distance_From_Convention_Center_Miles_x',
       u'Distance_From_City_Center_Miles_x', u'TYPE', u'sub_type', u'Address',
       u'ZipCode', u'Zip2', u'Zips', u'lat', u'long', u'Density_1_mile_ALL',
       u'Density_2_miles_ALL', u'Density_3_mile_ALL', u'Density_5_miles_ALL',
       u'Rating', u'Star', u'No_Of_Review', u'Certificate_of_Excellence',
       u'Recommended_length_of_visit', u'Fee', u'No_of_rooms',
       u'Source_

In [ ]:
import mysql.connector
cnx = mysql.connector.connect(user='puneet', password='root',host='192.157.250.82',database = 'testdb')
stmt = "CREATE TABLE t2(name VARCHAR(100),num int)"
cursor = cnx.cursor()
cursor.execute(stmt)
cnx.close()

CREATE TABLE barcelona(Source VARCHAR(1000),City_x VARCHAR(1000),CheckInDate VARCHAR(1000),CheckOutDate VARCHAR(1000),LOS int,Occupancy int,RoomType VARCHAR(1000),BoardType VARCHAR(1000),Hotel_Name_x VARCHAR(1000),Star_Rating float,Guest_Rating float,Guest_Recommendation VARCHAR(1000),Guest_Review_Count float,Strike_Rate float,Rate float,Rooms_Left int,Cancellation_Policy VARCHAR(1000),Discount VARCHAR(1000),Deal float,Page_Position int,Hotel_Rank int,Hotel_Overall_Rank int,Latitude_x float,Longitude_x float,Location VARCHAR(1000),Phone_Number VARCHAR(1000),Reward_Points int,Star_Rating_x float,Latitude_y float,Longitude_y float,Distance_From_Airport_Miles_x float,Distance_From_Convention_Center_Miles_x float,Distance_From_City_Center_Miles_x float,TYPE float,sub_type float,Address float,ZipCode float,Zip2 float,Zips float,lat float,lon float,Density_1_mile_ALL float,Density_2_miles_ALL float,Density_3_mile_ALL float,Density_5_miles_ALL float,Rating float,Star float,No_Of_Review float,